## Installations and Import Statements 


In [1]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 12.1 MB/s eta 0:00:00


In [2]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install dlib

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import cv2
import os 
import dlib
import numpy as np
from pathlib import Path
import pandas as pd
import os
from random import shuffle
from tqdm import tqdm_notebook
import itertools
import shutil
from matplotlib import pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.models import load_model
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')#made change
mixed_precision.set_policy(policy)

C:\Users\lycas02\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


ImportError: cannot import name 'experimental' from 'tensorflow.keras.mixed_precision' (C:\Users\lycas02\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\_tf_keras\keras\mixed_precision\__init__.py)

# Functions for Processing

In [ ]:
def get_input(folder):
    images = np.zeros((num_images,128,128,3), dtype = np.float32)
    imagespath = range(0,2054)
    for i in range(num_images):
        a= str(imagespath[i]).zfill(2)
        img = cv2.resize(plt.imread(folder+a+'.jpg'), (128, 128), interpolation=cv2.INTER_CUBIC)/255
        #print(a)
        images[i] = img
        
    return [images]

## CNN: Testing

In [ ]:
model_path = 'models/CNN50_Tom_Cruise/'
attempt = 0
foldername = model_path+"_xceptionimagenet_attempt"+str(attempt)+"/"
num_images = 2054
img_size = 128
modelname = foldername+"_xceptionimagenet_"+str(num_images)+"_"+str(img_size)
cnn_model = load_model(modelname+".h5")

In [ ]:
test_folder="../Tom_Cruise_Dataset/Tom_Cruise_dataset_two_faces/testing/real/TOM CRUISE 2020 Deepfake  VFX Breakdown  Side by Side_1080p_Trim/"
batch_input = get_input(test_folder)
batch_x = np.array(batch_input)
probr1 = cnn_model.predict(batch_x)
test_folder="../Tom_Cruise_Dataset/Tom_Cruise_dataset_two_faces/testing/fake/TOM CRUISE 2020 Deepfake  VFX Breakdown  Side by Side_1080p_Trim/"
batch_input = get_input(test_folder)
batch_x = np.array(batch_input)
probf1 = cnn_model.predict(batch_x)

# CNN:Reconstruction

In [ ]:
p = "../mmod_human_face_detector.dat"
cnn_face_detector = dlib.cnn_face_detection_model_v1(p)
for i in range(1):
    try:
        vidcap = cv2.VideoCapture('../Tom_Cruise/TOM CRUISE 2020 Deepfake VFX Breakdown Side by Side_1080p_Trim.mp4')
        success,image = vidcap.read()
        count = 0
        success = True
        frames_path="../Tom_Cruise/Completeframe/TOM_CRUISE_2020_Deepfake_VFX_Breakdown_Side_by_Side_1080p_Trim/"
        print("NOW FRAMING")
        imagespath = range(0,2241)
        kr=0
        kf=0
        prediction_folder_path='Tom_Cruise/Reconstructed/frames1/'
        Path(prediction_folder_path).mkdir(parents=True, exist_ok = True)
        real_count_real=0
        fake_count_real=0
        real_count_fake=0
        fake_count_fake=0
        print("NOW DETECTING")
        m=0
        #Perform face detection
        for it in range(0,2241):
            #try:
            a= str(imagespath[it]).zfill(4)
            image=cv2.imread(frames_path+a+'.jpg')
            
            if image is None:
                print("Could not read input image")
                continue

            faces_cnn = cnn_face_detector(image[:,:,::-1],0)
            
            if(len(faces_cnn)!=2):
                print("Total Faces =     " ,len(faces_cnn))
                cv2.imwrite(prediction_folder_path+a+'.jpg',image)
                continue
            else:
                odd=1
                for face in faces_cnn:
                    
                    x = face.rect.left()
                    y = face.rect.top()
                    w= face.rect.right() - x
                    h= face.rect.bottom() - y 
                    y1 =y-50
                    y2= y+h+30
                    x1=x-15
                    x2=x+w+50
                    
                    if odd==1:
                        odd=0
                        result=probr1[0,kr,0]
                        kr+=1
                        if(kr==2054):
                            print(a)
                        if(result<0.5):
                            print("real")
                            cv2.rectangle(image, (x1,y1), (x2,y2), (0,255,0), 2)
                            cv2.putText(image, 'REAL:'+str(result), (30, 250), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (35,102,11), thickness=2,lineType=cv2.LINE_AA) 
                            real_count_real+=1
                            
                        else:
                            print("fake")
                            cv2.rectangle(image, (x1,y1), (x2,y2), (0,0,255), 2)
                            cv2.putText(image, 'FAKE:'+str(result), (30, 250), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0,255), thickness=2,lineType=cv2.LINE_AA) 
                            fake_count_real+=1
                        real_count= real_count_real
                        fake_count= fake_count_real
                        cv2.putText(image, 'REAL:'+str(real_count), (30, 400), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (35,102,11),thickness=2, lineType=cv2.LINE_AA) 
                        cv2.putText(image, 'FAKE:'+str(fake_count), (30, 450), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0,255), thickness=2,lineType=cv2.LINE_AA) 
                        cv2.imwrite(prediction_folder_path+a+'.jpg',image)
                        
                    else:
                        odd=1
                        result=probf1[0,kf,0]
                        kf+=1
                        if(kf==2054):
                            print(a)
                        if(result<0.5):
                            print("real")
                            cv2.rectangle(image, (x1,y1), (x2,y2), (0,255,0), 2)
                            cv2.putText(image, 'REAL:'+str(result), (1600, 250), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (35,102,11), thickness=2,lineType=cv2.LINE_AA) 
                            real_count_fake+=1
                            
                        else:
                            print("fake")
                            cv2.rectangle(image, (x1,y1), (x2,y2), (0,0,255), 2)
                            cv2.putText(image, 'FAKE:'+str(result), (1600, 250), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0,255), thickness=2,lineType=cv2.LINE_AA) 
                            fake_count_fake+=1
                        real_count= real_count_fake
                        fake_count= fake_count_fake
                        cv2.putText(image, 'REAL:'+str(real_count), (1600, 400), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (35,102,11),thickness=2, lineType=cv2.LINE_AA) 
                        cv2.putText(image, 'FAKE:'+str(fake_count), (1600, 450), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0,255), thickness=2,lineType=cv2.LINE_AA) 
                        cv2.imwrite(prediction_folder_path+a+'.jpg',image)
                print(it)
        img_array = []
        for j in range(num_images):
            
            a= str(imagespath[j]).zfill(4)
            
            try:    
                img = cv2.imread(prediction_folder_path+a+'.jpg')
                height, width, layers = img.shape
                size = (width,height)
                img_array.append(img)
            except:
                continue
        print("NOW RECONSTRUCTING")
        save_path='Tom_Cruise/Reconstructed/rvideo/'
        Path(save_path).mkdir(parents=True, exist_ok = True)
        out = cv2.VideoWriter(save_path+'Tom_Cruise_video'+'.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 30, size)

        for i in range(len(img_array)):
            out.write(img_array[i])
        out.release()
        print("VIDEO COMPLETED:")
        
    except Exception as e:
        print(e)
        continue

## Video Deepfake Detection Using LSTM

Required Function:

In [ ]:
num_images = 20
img_size = 128

def get_input( folder):
    images = np.zeros((num_images,img_size,img_size,3), dtype = np.float32)
    imagespath = [f for f in os.listdir(folder+"/") if 'jpg' in f]
    #print(dataset+label+folder)
    for i in range(num_images):
        img = cv2.resize(plt.imread(folder+"/"+imagespath[i]), (img_size, img_size), interpolation=cv2.INTER_CUBIC)/255
        images[i] = img
    return [images]

CNN+ LSTM Testing

In [ ]:
test_folder1="../Tom_Cruise_Dataset/Tom_Cruise_dataset_two_faces/testing/fake/TOM CRUISE 2020 Deepfake  VFX Breakdown  Side by Side_1080p_Trim"
test_folder2="../Tom_Cruise_Dataset/Tom_Cruise_dataset_two_faces/testing/real/TOM CRUISE 2020 Deepfake  VFX Breakdown  Side by Side_1080p_Trim"

batch_input1=get_input(test_folder1)
batchx1=np.array(batch_input1)

batch_input2=get_input(test_folder2)
batchx2=np.array(batch_input2)

model=load_model("All_Models/Tom_Cruise/tom_cruise_xceptionimagenet_attempt1/_xceptionimagenet_2054_128_finetuned.h5")
result1=model.predict(batchx2)
print(result1)

result2=model.predict(batchx1)
print(result2)


CNN + LSTM Reconstruction

In [ ]:

video_folder="../Tom_Cruise/Completeframe/TOM_CRUISE_2020_Deepfake_VFX_Breakdown_Side_by_Side_1080p_Trim"

path_names=video_folder.split('/')
prediction_folder="Predictions/"+path_names[-1]+"/predicted_frames"
ssh = Path(prediction_folder)
ssh.mkdir(parents=True)

frames=os.listdir(video_folder+"/")
frames.sort()

if result1[0][0]>=0.5:
    print(result1)
    for i in range(0,2241):
        if frames[i] ==".ipynb_checkpoints":
            continue
        image=cv2.imread(video_folder+"/"+frames[i])
        if i<=216:
            destination=cv2.rectangle(image, (3,100), ((image.shape[1]//2)-20,image.shape[0]-100),(0,0,255), 5) 
        elif i<=290:
            destination=cv2.rectangle(image, (3,100), ((image.shape[1]//2)-20-50,image.shape[0]-100),(0,0,255), 5) 
        else:
            destination=cv2.rectangle(image, (3,100), ((image.shape[1]//2)-20+30,image.shape[0]-100),(0,0,255), 5) 
        #wide,length,_=(destination.shape)
        cv2.putText(destination, 'Fake:'+str(round(result1[0][0],4)), (30,image.shape[0]-100-35), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0,255),thickness=2, lineType=cv2.LINE_AA)
        cv2.imwrite(prediction_folder+"/"+frames[i],destination)
else:
    print(result1)
    for i in range(0,2241):
        if frames[i] ==".ipynb_checkpoints":
            continue
        image=cv2.imread(video_folder+"/"+frames[i])
        if i<=216:
            destination=cv2.rectangle(image, (3,100), ((image.shape[1]//2)-20,image.shape[0]-100),(0,255,0), 5) 
        elif i<=290:
            destination=cv2.rectangle(image, (3,100), ((image.shape[1]//2)-20-50,image.shape[0]-100),(0,255,0), 5) 
        else:
            destination=cv2.rectangle(image, (3,100), ((image.shape[1]//2)-20+30,image.shape[0]-100),(0,255,0), 5) 
        #wide,length,_=(destination.shape)
        cv2.putText(destination, 'Real:'+str(round(result1[0][0],4)), (30,image.shape[0]-100-35), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (35,102,11),thickness=2, lineType=cv2.LINE_AA)
        cv2.imwrite(prediction_folder+"/"+frames[i],destination)
if result2[0][0]>=0.5:
    print(result1)
    for i in range(0,2241):
        if frames[i] ==".ipynb_checkpoints":
            continue
        image=cv2.imread(prediction_folder+"/"+frames[i])
        if i<=216:
            destination=cv2.rectangle(image,  ((image.shape[1]//2)-15,100),((image.shape[1])-3,image.shape[0]-100),(0,0,255), 5) 
        elif i<=290:
            destination=cv2.rectangle(image,  ((image.shape[1]//2)-20-45,100),((image.shape[1])-3,image.shape[0]-100),(0,0,255), 5) 
        else:
            destination=cv2.rectangle(image, ((image.shape[1]//2)-20+35,100),((image.shape[1])-3,image.shape[0]-100),(0,0,255), 5) 
        wide,length,_=(destination.shape)
        cv2.putText(destination, 'Fake:'+str(round(result2[0][0],4)), ((image.shape[1])-330,wide-100-35), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,0,255),thickness=2, lineType=cv2.LINE_AA)
        cv2.imwrite(prediction_folder+"/"+frames[i],destination)
else:
    print(result2)
    print(result1)
    for i in range(0,2241):
        if frames[i] ==".ipynb_checkpoints":
            continue
        image=cv2.imread(prediction_folder+"/"+frames[i])
        if i<=216:
            destination=cv2.rectangle(image,((image.shape[1]//2)-15,100),((image.shape[1])-3,image.shape[0]-100),(0,255,0), 5) 
        elif i<=290:
            destination=cv2.rectangle(image,((image.shape[1]//2)-20-45,100),((image.shape[1])-3,image.shape[0]-100),(0,255,0), 5) 
        else:
            destination=cv2.rectangle(image,((image.shape[1]//2)-20+35,100),((image.shape[1])-3,image.shape[0]-100),(0,255,0), 5) 
        wide,length,_=(destination.shape)
        cv2.putText(destination, 'Real:'+str(round(result2[0][0],4)), ((image.shape[1])-330,wide-100-35), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (0,255,0),thickness=2, lineType=cv2.LINE_AA)
        cv2.imwrite(prediction_folder+"/"+frames[i],destination)

        
images=[]
files=os.listdir(prediction_folder)
files.sort()
print(files)

for filename in files:
    try:    
        img = cv2.imread(prediction_folder+"/"+filename)
        height, width, layers = img.shape
        height, width, layers = img.shape
        size = (width,height)
        images.append(img)
    except:
        continue
        
print("NOW RECONSTRUCTING---")
    
out = cv2.VideoWriter("Predictions/"+path_names[-1]+'/predicted_video30.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 30, size)

for i in range(len(images)):
    out.write(images[i])

out.release()

print("VIDEO COMPLETED!!!")




## Video Deepfake Detection Using GRU

In [ ]:
epochs = 40
batch_size = 16

train_spe = len(os.listdir(dataset+"/"+'real'))//batch_size
print(train_spe)
val_spe = len(os.listdir(dataset+"/"+'real'))//batch_size
print(val_spe)
test_spe = len(os.listdir(dataset+"/"+'real'))//(batch_size)
print(test_spe)

print('For GRU:')
train_datagen = video_data_generator(dataset,'train', batch_size = batch_size)
val_datagen = video_data_generator(dataset,'val', batch_size = batch_size)
test_datagen = video_data_generator(dataset,'test', batch_size = batch_size)
print("For model without fine-tuning validation loss and validation accuracy is:")
cnngru_model = load_model('models/TDCNN_GRU_20_Epochs/multiframe_full_xceptionimagenet_attempt2/multiframe_full_xceptionimagenet_10_128.h5')
test_datagen = video_data_generator(dataset,'test', batch_size = batch_size)
start_time1 = time.clock()
print(cnngru_model.evaluate(test_datagen,steps=2*test_spe,verbose=0))
current_time1= time.clock()
print("Evaluation time required without fine tuning:")
print(current_time1-start_time1)

yhat_classes=cnngru_model.predict(test_datagen,steps=2*test_spe,verbose=0)
accuracy = accuracy_score(testy, yhat_classes.round())
print('Accuracy:   ',accuracy)
# precision tp / (tp + fp)
precision = precision_score(testy, yhat_classes.round())
print("Precision:  ",precision)
# recall: tp / (tp + fn)
recall = recall_score(testy, yhat_classes.round())
print('Recall:     ',recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(testy, yhat_classes.round())
print('f1 score:   ',f1)


print("For model with fine-tuning validation loss and validation accuracy is:")
cnngru_model = load_model('models/TDCNN_GRU_20_Epochs/multiframe_full_xceptionimagenet_attempt2/multiframe_full_xceptionimagenet_10_128_finetuned.h5')
test_datagen = video_data_generator(dataset,'test', batch_size = batch_size)
start_time_1 = time.clock()
print(cnngru_model.evaluate(test_datagen,steps=2*test_spe,verbose=0))
current_time_1 = time.clock()
print("Evaluation time required with fine tuning:")
print(current_time_1-start_time_1)

yhat_classes=cnngru_model.predict(test_datagen,steps=2*test_spe,verbose=0)
accuracy = accuracy_score(testy, yhat_classes.round())
print('Accuracy:   ',accuracy)
# precision tp / (tp + fp)
precision = precision_score(testy, yhat_classes.round())
print("Precision:  ",precision)
# recall: tp / (tp + fn)
recall = recall_score(testy, yhat_classes.round())
print('Recall:     ',recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(testy, yhat_classes.round())
print('f1 score:   ',f1)